In [ ]:
# default_exp utils

In [ ]:
#export
import matplotlib.pyplot as plt
import PIL
import torch
import gc
import os
from torch.utils.tensorboard import SummaryWriter
import datetime
from functools import partial

import ipywidgets as widgets
from IPython.display import clear_output
import math
import torchvision

# Utils

## Divers

In [ ]:
#export
def clean():
    gc.collect()
    gc.collect()

In [ ]:
#export
def extract_minibatch(dl):
    minibatch = next(iter(dl))
    print(f'minibatch extracted. Shape: {minibatch.shape}')
    return minibatch

## Files utils

In [ ]:
#export

def get_files_dir(path, ext, debug=False):
    items = []
    for r, d, f in os.walk(path):
            if debug: print(f'In directory {r}: {len(f)} files')
            for file in f:
                for e in ext:
                    if e in file:
                        items.append(r+'/'+file)
                    if debug: print(f'working on img {path+file}')
    return list(set(items))


def get_name_training(prefix, placeholder=True, params={}):
    id_training = get_id_training(placeholder=placeholder, params=params)
    return f'{id_training}-{prefix}'


def get_id_training(path='./svg', name='', placeholder=True, params={}):
    files = get_files_dir(path, [''], debug=False)
    
    index = 0
    
    for f in files:
        try:
            number = int(f.split('/')[-1].split('-')[0])
        except ValueError:
            continue
        
        if number > index:
            index = number
            
    print(f'Biggest index = {index}, new index = {index+1}')
    if placeholder:
        with open(f'{path}/{index+1}-placeholder-{name}', 'w') as fp:
            fp.write(f'{params}')
    
    
    return index+1

## Transforms

In [ ]:
#export

def to_byte_tensor(item):
    res = torch.ByteTensor(torch.ByteStorage.from_buffer(item.tobytes()))
    w,h = item.size
    return res.view(h,w,-1).permute(2,0,1)

def to_float_tensor(item): return item.float().div_(255.)


def normalize_tan(x):
    return (x - 0.5) / 0.5

def decode_from_tan(x):
    return (x * 0.5) + 0.5

## Visualization

In [ ]:
#export

def loader(path, tfms, show_img=False):
    style_im_raw = PIL.Image.open(path).convert('RGB')
    if show_img:
        plt.imshow(style_im_raw)
    
    im = tfms(style_im_raw)
    
    show_img_from_tensor(im)
    return im.unsqueeze(0).cuda()

def show_img_from_tensor(x, ax=None, decoder=None, figsize=(8,8), title=''):
    if decoder is not None:
        x = decoder(x)
    img = x.detach().cpu().numpy().transpose((1,2,0))
    
    
    
    if ax:
        ax.imshow(img)
    else:
        fig, ax = plt.subplots(figsize=figsize)
        ax.imshow(img)
        
        
        if len(title) > 0:
            fig.suptitle(title, y=0.9, fontsize=15)

In [ ]:
#export

def gradient_flow(m):
    mean_grad = []
    std_grad = []
    median_grad = []
    norm_grad = []
    names = []
    for i, (n, p) in enumerate(m.named_parameters()):
        if p.grad is not None and p.grad.ndimension() > 1 :
            #print(n, p.grad.shape, p.grad.ndimension())
            mean_grad.append(p.grad.mean())
            std_grad.append(p.grad.std())
            median_grad.append(p.grad.median())
            norm_grad.append(p.grad.norm())
            names.append(n.replace('layers', 'l'))
    
    return names, mean_grad, std_grad, median_grad, norm_grad

def viz_grad_flow(names, mean_grad, std_grad, median_grad, norm_grad, target= 'Not specified'):
    fig, axes = plt.subplots(nrows=3, ncols=1,figsize=(15, 12))
    
    axes = axes.flatten()
    
    axes[0].plot(names, mean_grad, c='r')
    axes[0].plot(names, median_grad, c='b')
    axes[1].plot(names, std_grad)
    axes[2].plot(names, norm_grad)

    
    axes[0].set_title(f'Mean(R)/Med(B) {target}')
    axes[1].set_title(f'Std {target}')
    axes[2].set_title(f'Norm {target}')
    
    
    return fig

In [ ]:
#export
class OutputTrainingWidget:
    def __init__(self, batch_size = 64, history_size=1):
        self.display = widgets.Output(layout={'border': '1px solid black'})
        self.batch_size = batch_size
        self.nrow_batch = math.ceil(math.sqrt(self.batch_size))
        self.history_size = history_size
        self.current = 0
        
    def display_widget(self):
        display(self.display)

    def display_minibatch(self, minibatch, nb_iter=0, normalize=True):
        self.current += 1
        with self.display:
            plt.figure(figsize=(20,20))
            if self.history_size != -1 and self.current % self.history_size == 0:
                clear_output()
            print(f'Display minibatch for iter {nb_iter}')

            ax = plt.gca()
            
            grid = torchvision.utils.make_grid(minibatch[:,:3,:,:], nrow=self.nrow_batch, normalize=normalize)
            show_img_from_tensor(grid, ax = ax)
            plt.show()

<function torchvision.utils.make_grid(tensor, nrow=8, padding=2, normalize=False, range=None, scale_each=False, pad_value=0)>

### Tensorboard

In [ ]:
#export

def get_tb_logdir(name):
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    return 'logs/' + current_time + f'/{name}'

def get_identity_training(name):
    return f'{name}-{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
    
def get_tb_writer(name):

    train_summary_writer = SummaryWriter(get_tb_logdir(name))
    
    return train_summary_writer

# Generate

In [ ]:
from nbdev.export import *
notebook2script()


Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted 02_training.ipynb.
Converted index.ipynb.
